-----------
- # 요약
    - ## 로지스틱 회귀
        - ### 이진 분류
            - 과정
            - nn.Module 적용 시 변경점
     
-----------

# 로지스틱 회귀
## 이진 분류
- 주어진 데이터를 1,0 출력으로 두개의 클래스로 분리하는 문제임
- 활성화 함수는 시그모이드, 비용함수는 이진 교차 엔트로피 손실 사용


In [2]:
## 모듈 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
## 훈련 데이터
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [ ]:
## 초기 신경망 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

## 시그모이드 하드코딩
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))
print(hypothesis)
print(hypothesis.shape)

## 시그모이드 함수 사용
hypothesis = torch.sigmoid(x_train.matmul(W) + b)  # or .mm or @
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])
tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


In [ ]:
## 손실함수 정의
## 행렬 형태에 주의할 것
## Binary Cross-entropy Loss
-(
    y_train[0] * torch.log(hypothesis[0])
    + (1 - y_train[0]) * torch.log(1 - hypothesis[0])
)

losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

cost = losses.mean()
print(cost)

# 미리 정의된 F 손실함수 사용시
F.binary_cross_entropy(hypothesis, y_train)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)
tensor(0.6931, grad_fn=<MeanBackward0>)


tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

# 전체코드
## Naive
1. 데이터 로드
2. 옵티마이저 초기화
3. for 루프
    1. 신경망 순전파
    2. 손실함수 계산
    3. 그레디언트 초기화 후 역전파
    4. 그레디언트 갱신
5. 정확도 계산

In [ ]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)  # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print(
            "Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%".format(
                epoch,
                nb_epochs,
                cost.item(),
                accuracy * 100,
            )
        )

Epoch    0/1000 Cost: 0.693147 Accuracy 50.00%
Epoch  100/1000 Cost: 0.134722 Accuracy 100.00%
Epoch  200/1000 Cost: 0.080643 Accuracy 100.00%
Epoch  300/1000 Cost: 0.057900 Accuracy 100.00%
Epoch  400/1000 Cost: 0.045300 Accuracy 100.00%
Epoch  500/1000 Cost: 0.037261 Accuracy 100.00%
Epoch  600/1000 Cost: 0.031672 Accuracy 100.00%
Epoch  700/1000 Cost: 0.027556 Accuracy 100.00%
Epoch  800/1000 Cost: 0.024394 Accuracy 100.00%
Epoch  900/1000 Cost: 0.021888 Accuracy 100.00%
Epoch 1000/1000 Cost: 0.019852 Accuracy 100.00%


## nn.Module 사용시
- 모듈 정의 부분 주의해서 볼것

In [ ]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))


model = BinaryClassifier()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)#####

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)#####

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print(
            "Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%".format(
                epoch,
                nb_epochs,
                cost.item(),
                accuracy * 100,
            )
        )

Epoch    0/100 Cost: 1.821371 Accuracy 50.00%
Epoch   10/100 Cost: 0.613015 Accuracy 50.00%
Epoch   20/100 Cost: 0.459853 Accuracy 66.67%
Epoch   30/100 Cost: 0.388596 Accuracy 83.33%
Epoch   40/100 Cost: 0.329732 Accuracy 83.33%
Epoch   50/100 Cost: 0.278133 Accuracy 83.33%
Epoch   60/100 Cost: 0.231362 Accuracy 100.00%
Epoch   70/100 Cost: 0.190611 Accuracy 100.00%
Epoch   80/100 Cost: 0.162033 Accuracy 100.00%
Epoch   90/100 Cost: 0.146851 Accuracy 100.00%
Epoch  100/100 Cost: 0.136572 Accuracy 100.00%
